### * Домашнее задание: Многоклассовая классификация

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from hyperopt import hp, tpe, space_eval
from hyperopt.fmin import fmin



In [0]:

import itertools
import warnings
warnings.simplefilter('ignore')

## Предобработка данных

In [0]:
# Загрузим набор данных

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

In [0]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

In [0]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [0]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [0]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

In [0]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).

In [0]:
list(df.VehUsage.unique())

In [0]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

Фактор "SocioCateg" содержит информацию о социальной категории в виде кодов классификации CSP. Агрегируем имеющиеся коды до 1 знака, а затем закодируем их с помощью one-hot encoding.

[Wiki](https://fr.wikipedia.org/wiki/Professions_et_cat%C3%A9gories_socioprofessionnelles_en_France#Cr%C3%A9ation_de_la_nomenclature_des_PCS)

[Более подробный классификатор](https://www.ast74.fr/upload/administratif/liste-des-codes-csp-copie.pdf)

In [0]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [0]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

In [0]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [0]:
df = df.select_dtypes(exclude=['object'])

Также создадим такую переменную, как квадрат возраста.

In [0]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

Для моделирования частоты убытков сгенерируем показатель как сумму индикатора того, что убыток произошел ("ClaimInd") и количества заявленных убытков по различным видам ущерба за 4 предшествующих года ("ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen").

В случаях, если соответствующая величина убытка равняется нулю, сгенерированную частоту также обнулим.

In [0]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [0]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

In [0]:
# import plotly.express as px
# fig = px.bar(df, x='ClaimsCount', y='ClaimAmount', title='Зависимость между частотой и величиной убытков')
# fig.show()

Добавляется  признак среднего убытка, как отношение величины убытков к их частоте.

In [0]:
dfAC = df[df.ClaimsCount > 0].copy()
dfAC['AvgClaim'] = dfAC.ClaimAmount/dfAC.ClaimsCount

XGBoost для многоклассовой классификации принимает на вход значения меток классов в виде `[0, num_classes]`. Заменим значение 11 на 10.

In [0]:
df['ClaimsCount']=df['ClaimsCount'].replace(to_replace = 11., value =10.) 

df['ClaimsCount'].unique()

Заметим, что в данном случае присутствует проблема несбалансированности классов. Поэтому, для того, чтобы по возможности избежать ее, воспользуемся взвешиванием наблюдений для обучения модели. Для этого в исходном наборе данных создадим столбец `weight`. Присвоим ему некоторые значения, например, можно задать `0.05` для значений `ClaimsCount` 0, а для остальных - 1 (Для этого можем использовать функцию `np.where`). Также можно попробовать какой-либо другой способ задания весов, приведенный пример не гарантирует хороших результатов.

In [0]:

df['weight']= ((df['ClaimsCount'] / df['ClaimsCount'].sum())**1.2)*100000
df.loc[df.ClaimsCount.isin([0]), 'weight'] = 0.05
df.tail(10)

# df_group_count['weight'] = df_group_count['ClaimsCount'].where(df_group_count['ClaimsCount'] < 1., 1.)
# df_group_count.head()
# weights = class_weight.compute_class_weight('balanced',
#                                             df_f.ClaimsCount.unique(),
#                                              df_f.ClaimsCount)
# def make_weight (row):
#    return weights[int(row['ClaimsCount'])]
# df_f['weight'] = df_f.apply (lambda row: make_weight(row), axis=1)



## Разделение набора данных на обучающую, валидационную и тестовую выборки

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = df.drop(columns=['Exposure', 'ClaimsCount'])
y = df['ClaimsCount']


Разобьем имеющийся набор данных на обучающую, валидационную и тестовую выборки в отношениях 70%/15%/15% соответственно. Зададим зерно для случайного разбиения равным 10.

In [0]:
# Разбиение датасета для частоты на train/val/test

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=10)

Далее, создадим объекты `DMatrix` для обучающей, валидационной и тестовой выборок. Для обучающей выборки также укажем параметр `weight` равным полученному ранее столбцу весов. Данный столбец также нужно исключить из объекта передаваемого в параметр `data`.

In [0]:
xgb_train = xgb.DMatrix(x_train.drop(columns=['weight']).values,
                        y_train.values,
                        weight=x_train['weight'].values)

xgb_valid = xgb.DMatrix(x_valid.drop(columns=['weight']).values,
                        y_valid.values)

xgb_test = xgb.DMatrix(x_test.drop(columns=['weight']).values,
                       y_test.values)

In [0]:
def objective(params):
    parameters = {
        'objective': 'multi:softmax',
        'max_depth':  int(params['max_depth']),
        'min_child_weight': params['min_child_weight'],
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'eta': params['eta'],
        'alpha': params['alpha'],
        'lambda': params['lambda'],
        'num_class': y.nunique()
    }
    
    cv_result = xgb.cv(parameters, xgb_train, num_boost_round=int(params['num_boost_round']),
                       nfold=5, metrics=['merror'], seed=21, maximize=False, early_stopping_rounds=10)
    score = cv_result['test-merror-mean'][-1:].values[0]
    return score

In [0]:
space = {'num_boost_round': hp.quniform('num_boost_round', 50, 500, 1),
         'max_depth': hp.choice('max_depth', [5, 8, 10, 12, 15]),
         'min_child_weight': hp.uniform('min_child_weight', 0, 50),
         'subsample': hp.uniform('subsample', 0.5, 1),
         'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
         'alpha': hp.uniform('alpha', 0.8, 1),
         'lambda': hp.uniform('lambda', 0, 1),
         'eta': hp.uniform('eta', 0.01, 1),
         'num_class': 20
        }


In [0]:
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)

In [0]:
best_params = space_eval(space, best)
best_params['objective'] = 'multi:softmax'
best_params

# Применение модели

In [0]:
progress = dict()
xgb_model = xgb.train(params=best_params, dtrain=xgb_train,
                        maximize=False, evals=[(xgb_train, "train"),(xgb_valid, "valid")],
                        early_stopping_rounds=10, evals_result=progress, verbose_eval=False)

# Оценка результатов


In [0]:


dfsets = [{'set': 'train', 'dmat': xgb_train, 'target': y_train},
          {'set': 'valid', 'dmat': xgb_valid, 'target': y_valid},
          {'set': 'test', 'dmat': xgb_test, 'target': y_test}]
for dfset in dfsets:
    class_preds = xgb_model.predict(dfset['dmat'])
    print('F1 Score on ' + str(dfset['set'])+':', f1_score(dfset['target'], class_preds, average='micro')) 
    print('Accuracy on ' + str(dfset['set'])+':', accuracy_score(dfset['target'], class_preds)) 
    print('')



In [0]:
plt.subplots(1,3, figsize=(15,3))
for i in range(len(dfsets)):
    confmatrix = confusion_matrix(dfsets[i]['target'], xgb_model.predict(dfsets[i]['dmat']))
    plt.subplot(1,3,i+1)
    plt.imshow(confmatrix, cmap='Greys')
    plt.colorbar()
    plt.ylabel('True')
    plt.xlabel('Predicted')
plt.show()


Как вы оцениваете качество построенной модели? Какие проблемы могут здесь присутствовать? Как можно улучшить результат?


Подозрительно хорошие результаты на тесте.
Где ошибка- не пойму пока.


# Drafft

In [0]:
# pip install hyperopt

In [0]:
# Отбор признаков (Feature Importance)

importance_type = ['total_gain', 'gain', 'weight', 'total_cover', 'cover']
xgb.plot_importance(xgb_model, importance_type=importance_type[0]); plt.show()

In [0]:
# Отбор признаков (Feature Importance)

importance_type = ['total_gain', 'gain', 'weight', 'total_cover', 'cover']
xgb.plot_importance(xgb_model, importance_type=importance_type[0]); plt.show()

In [0]:
!pip install eli5

In [0]:
import eli5

In [0]:
eli5.show_weights(xgb_model)